In [88]:
import pandas as pd

dataset_name = "XSum"
# Treatment
df_deepseek_1_5b = pd.read_csv(f"./{dataset_name}/Treatment/results_deepseek-ai_DeepSeek-R1-Distill-Qwen-1.5B_{dataset_name}.csv")

df_gpt_neo_125m = pd.read_csv(f"./{dataset_name}/Treatment/results_EleutherAI_gpt-neo-125m_{dataset_name}.csv")
df_gpt_neo_1_3b = pd.read_csv(f"./{dataset_name}/Treatment/results_EleutherAI_gpt-neo-1.3B_{dataset_name}.csv")
df_gpt_neo_2_7b = pd.read_csv(f"./{dataset_name}/Treatment/results_EleutherAI_gpt-neo-2.7B_{dataset_name}.csv")

df_opt_125m = pd.read_csv(f"./{dataset_name}/Treatment/results_facebook_opt-125m_{dataset_name}.csv")
df_opt_1_3b = pd.read_csv(f"./{dataset_name}/Treatment/results_facebook_opt-1.3b_{dataset_name}.csv")
# df_opt_350m = pd.read_csv(f"./{dataset_name}/Treatment/results_facebook_opt-350m_{dataset_name}.csv")
df_opt_2_7b = pd.read_csv(f"./{dataset_name}/Treatment/results_facebook_opt-2.7b_{dataset_name}.csv")
df_opt_6_7b = pd.read_csv(f"./{dataset_name}/Treatment/results_facebook_opt-6.7b_{dataset_name}.csv")

df_vicuna_7b = pd.read_csv(f"./{dataset_name}/Treatment/results_lmsys_vicuna-7b-v1.5_{dataset_name}.csv")

# Control
df_smol_1_7b = pd.read_csv(f"./{dataset_name}/Control/results_HuggingFaceTB_SmolLM2-1.7B-Instruct_{dataset_name}.csv")
df_smol_135m = pd.read_csv(f"./{dataset_name}/Control/results_HuggingFaceTB_SmolLM2-135M-Instruct_{dataset_name}.csv")
df_smol_360m = pd.read_csv(f"./{dataset_name}/Control/results_HuggingFaceTB_SmolLM2-360M-Instruct_{dataset_name}.csv")

df_phi_3_5 = pd.read_csv(f"./{dataset_name}/Control/results_microsoft_Phi-3.5-mini-instruct_{dataset_name}.csv")


In [89]:
dfs = {
    "df_deepseek_1_5b": df_deepseek_1_5b,
    "df_gpt_neo_1_3b": df_gpt_neo_1_3b,
    "df_gpt_neo_125m": df_gpt_neo_125m,
    "df_gpt_neo_2_7b": df_gpt_neo_2_7b,
    "df_opt_1_3b": df_opt_1_3b,
    "df_opt_125m": df_opt_125m,
    "df_opt_2_7b": df_opt_2_7b,
    # "df_opt_350m": df_opt_350m,
    "df_opt_6_7b": df_opt_6_7b,
    "df_vicuna_7b": df_vicuna_7b,
    "df_smol_1_7b": df_smol_1_7b,
    "df_smol_135m": df_smol_135m,
    "df_smol_360m": df_smol_360m,
    "df_phi_3_5": df_phi_3_5
}

In [90]:
# df_deepseek_1_5b.head()
len(df_gpt_neo_1_3b)

1701

In [91]:
from scipy.stats import ttest_ind

# Perform Welch’s t-test; return a dictionary mapping score names to significance (1 or 0)
def t_test(df_suspicious_name, df_control_name, score_names):
    df_suspicious, df_control = dfs[df_suspicious_name], dfs[df_control_name]
    score_sig = dict()
    for score_name in score_names:
        scores_suspicious, scores_control = df_suspicious[score_name], df_control[score_name]
        stat, p_value = ttest_ind(scores_suspicious, scores_control, equal_var=False)
        print(f"comparing {score_name} between {df_suspicious_name} and {df_control_name}")
        print(f"mean ({df_suspicious_name}) = {scores_suspicious.mean()}, mean ({df_control_name}) = {scores_control.mean()}")
        print(f"t-statistic: {stat:.3f}, p-value: {p_value:.5f}")
        if p_value <= 0.05:
            print("Significant difference between suspicious and control groups.")
            score_sig[score_name] = f"1, {round(scores_suspicious.mean(), 4)}, {round(scores_control.mean(), 4)}"
        else:
            print("No significant difference detected.")
            score_sig[score_name] = "0"
        print("----------------------------------------------------------------")
    return score_sig


In [92]:
score_names = df_deepseek_1_5b.drop(['Unnamed: 0', 'Model', 'Task_Prefix', 'Dataset_Name', 'Model_Responses', 'Gold_Labels'], axis=1).columns
score_names = list(score_names)

In [93]:
gpt = ["df_gpt_neo_125m", "df_gpt_neo_1_3b", "df_gpt_neo_2_7b"]
# opt = ["df_opt_125m", "df_opt_350m", "df_opt_1_3b", "df_opt_2_7b", "df_opt_6_7b"]
opt = ["df_opt_125m", "df_opt_1_3b", "df_opt_2_7b", "df_opt_6_7b"]
vicuna = ["df_vicuna_7b"]
smol = ["df_smol_135m", "df_smol_360m", "df_smol_1_7b"]
phi = ["df_phi_3_5"]
treatment = gpt + opt + vicuna
control = smol + phi

In [94]:
import itertools
# doing t-test for every possible combination 
all_pairs = list(itertools.product(treatment, control))

total = 0 # count total model pairs
model_pair_analysis = [] # store all model pairs with resulting score indications

for suspicious, control in all_pairs: 
    model_pair_analysis.append((
        suspicious,
        control,
        t_test(suspicious, control, score_names)
    ))
    total+= 1

comparing cos_sim_scores between df_gpt_neo_125m and df_smol_135m
mean (df_gpt_neo_125m) = 0.2837392103285181, mean (df_smol_135m) = 0.275623557275188
t-statistic: 1.341, p-value: 0.18012
No significant difference detected.
----------------------------------------------------------------
comparing levenshtein_distance between df_gpt_neo_125m and df_smol_135m
mean (df_gpt_neo_125m) = 97.11934156378601, mean (df_smol_135m) = 102.91240446796003
t-statistic: -6.282, p-value: 0.00000
Significant difference between suspicious and control groups.
----------------------------------------------------------------
comparing rouge1_precision between df_gpt_neo_125m and df_smol_135m
mean (df_gpt_neo_125m) = 0.13017792478245246, mean (df_smol_135m) = 0.1045275028261191
t-statistic: 8.119, p-value: 0.00000
Significant difference between suspicious and control groups.
----------------------------------------------------------------
comparing rouge1_recall between df_gpt_neo_125m and df_smol_135m
mean 

In [95]:
# from collections import defaultdict

# score_proportion = defaultdict(int) # store count of total model pairs where score S is significantly difference
# score_sig_match = defaultdict(list) # store pairs that contribute to the counts above^

# for sus, con, vals in model_pair_analysis:
#     for m in vals.keys(): 
#         score_proportion[m] += vals[m]
#         if vals[m]:
#             score_sig_match[m].append((sus, con))

In [96]:
# model size wise comparison
def model_size_wise_compare(model_list):
    df_model = pd.DataFrame()
    for i in range(len(model_list)):
        for j in range(i+1, len(model_list)):
            score_sig = t_test(model_list[i], model_list[j], score_names)
            print(score_sig)
            df_model = pd.concat([df_model, pd.DataFrame(score_sig, index=[f"{model_list[i]} vs {model_list[j]}"])])
            print()
    return df_model
    
# gpt-neo
df_gpt_neo = model_size_wise_compare(gpt)

comparing cos_sim_scores between df_gpt_neo_125m and df_gpt_neo_1_3b
mean (df_gpt_neo_125m) = 0.2837392103285181, mean (df_gpt_neo_1_3b) = 0.2935827904592883
t-statistic: -1.709, p-value: 0.08754
No significant difference detected.
----------------------------------------------------------------
comparing levenshtein_distance between df_gpt_neo_125m and df_gpt_neo_1_3b
mean (df_gpt_neo_125m) = 97.11934156378601, mean (df_gpt_neo_1_3b) = 96.65608465608466
t-statistic: 0.544, p-value: 0.58620
No significant difference detected.
----------------------------------------------------------------
comparing rouge1_precision between df_gpt_neo_125m and df_gpt_neo_1_3b
mean (df_gpt_neo_125m) = 0.13017792478245246, mean (df_gpt_neo_1_3b) = 0.1365820764155135
t-statistic: -2.090, p-value: 0.03673
Significant difference between suspicious and control groups.
----------------------------------------------------------------
comparing rouge1_recall between df_gpt_neo_125m and df_gpt_neo_1_3b
mean (df_

In [97]:
df_gpt_neo

,cos_sim_scores,levenshtein_distance,rouge1_precision,rouge1_recall,rouge1_f1,rouge2_precision,rouge2_recall,rouge2_f1,rougeL_precision,rougeL_recall,rougeL_f1,Min_10.0%_Prob,Min_20.0%_Prob,Min_30.0%_Prob,Min_40.0%_Prob,Min_50.0%_Prob
df_gpt_neo_125m vs df_gpt_neo_1_3b,0,0,"1, 0.1302, 0.1366","1, 0.1802, 0.1926","1, 0.1482, 0.1566",0,0,0,"1, 0.1036, 0.1087","1, 0.1429, 0.1529","1, 0.1177, 0.1244",0,0,0,"1, 7.1658, 6.0033","1, 6.4503, 5.3126"
df_gpt_neo_125m vs df_gpt_neo_2_7b,"1, 0.2837, 0.2988",0,"1, 0.1302, 0.1374","1, 0.1802, 0.1922","1, 0.1482, 0.1571","1, 0.016, 0.0189",0,"1, 0.0185, 0.0216",0,"1, 0.1429, 0.1513","1, 0.1177, 0.1235",0,0,0,"1, 7.1658, 5.7774","1, 6.4503, 5.094"
df_gpt_neo_1_3b vs df_gpt_neo_2_7b,0,0,0,0,0,0,0,0,0,0,0,0,"1, 7.7829, 7.5518","1, 6.8064, 6.5765","1, 6.0033, 5.7774","1, 5.3126, 5.094"


In [98]:
import os
os.makedirs(f"t_test/{dataset_name}")
df_gpt_neo.to_csv(f"t_test/{dataset_name}/gpt_neo_.csv")

In [99]:
# model size wise comparison
# opt
df_opt = model_size_wise_compare(opt)
df_opt.to_csv(f"t_test/{dataset_name}/opt.csv")

comparing cos_sim_scores between df_opt_125m and df_opt_1_3b
mean (df_opt_125m) = 0.2684157092748322, mean (df_opt_1_3b) = 0.23814407915100325
t-statistic: 4.994, p-value: 0.00000
Significant difference between suspicious and control groups.
----------------------------------------------------------------
comparing levenshtein_distance between df_opt_125m and df_opt_1_3b
mean (df_opt_125m) = 99.11052322163434, mean (df_opt_1_3b) = 99.22104644326866
t-statistic: -0.123, p-value: 0.90214
No significant difference detected.
----------------------------------------------------------------
comparing rouge1_precision between df_opt_125m and df_opt_1_3b
mean (df_opt_125m) = 0.12299591133167666, mean (df_opt_1_3b) = 0.11463933709796462
t-statistic: 2.640, p-value: 0.00832
Significant difference between suspicious and control groups.
----------------------------------------------------------------
comparing rouge1_recall between df_opt_125m and df_opt_1_3b
mean (df_opt_125m) = 0.168501601488480

In [100]:
df_opt

,cos_sim_scores,levenshtein_distance,rouge1_precision,rouge1_recall,rouge1_f1,rouge2_precision,rouge2_recall,rouge2_f1,rougeL_precision,rougeL_recall,rougeL_f1,Min_10.0%_Prob,Min_20.0%_Prob,Min_30.0%_Prob,Min_40.0%_Prob,Min_50.0%_Prob
df_opt_125m vs df_opt_1_3b,"1, 0.2684, 0.2381",0,"1, 0.123, 0.1146","1, 0.1685, 0.1592","1, 0.1391, 0.1304",0,0,0,"1, 0.0963, 0.0903",0,"1, 0.1088, 0.1026",0,0,"1, 7.5302, 6.5032","1, 6.6943, 5.7255","1, 5.9835, 5.0571"
df_opt_125m vs df_opt_2_7b,"1, 0.2684, 0.2209",0,"1, 0.123, 0.1088","1, 0.1685, 0.1489","1, 0.1391, 0.1234",0,0,0,"1, 0.0963, 0.0863","1, 0.1321, 0.1182","1, 0.1088, 0.0979",0,0,"1, 7.5302, 6.3684","1, 6.6943, 5.5888","1, 5.9835, 4.9229"
df_opt_125m vs df_opt_6_7b,"1, 0.2684, 0.2384",0,"1, 0.123, 0.1077","1, 0.1685, 0.154","1, 0.1391, 0.1242",0,0,0,"1, 0.0963, 0.0844","1, 0.1321, 0.1205","1, 0.1088, 0.0972",0,0,"1, 7.5302, 6.203","1, 6.6943, 5.4209","1, 5.9835, 4.7559"
df_opt_1_3b vs df_opt_2_7b,"1, 0.2381, 0.2209",0,0,"1, 0.1592, 0.1489","1, 0.1304, 0.1234",0,0,0,0,"1, 0.1256, 0.1182",0,0,"1, 7.4505, 7.3182","1, 6.5032, 6.3684","1, 5.7255, 5.5888","1, 5.0571, 4.9229"
df_opt_1_3b vs df_opt_6_7b,0,0,"1, 0.1146, 0.1077",0,0,0,0,0,"1, 0.0903, 0.0844",0,"1, 0.1026, 0.0972",0,"1, 7.4505, 7.1779","1, 6.5032, 6.203","1, 5.7255, 5.4209","1, 5.0571, 4.7559"
df_opt_2_7b vs df_opt_6_7b,"1, 0.2209, 0.2384","1, 98.4521, 100.4738",0,0,0,0,0,0,0,0,0,0,"1, 7.3182, 7.1779","1, 6.3684, 6.203","1, 5.5888, 5.4209","1, 4.9229, 4.7559"


In [101]:
df_smol = model_size_wise_compare(smol)
df_smol.to_csv(f"t_test/{dataset_name}/smol.csv")

comparing cos_sim_scores between df_smol_135m and df_smol_360m
mean (df_smol_135m) = 0.275623557275188, mean (df_smol_360m) = 0.3637882205138891
t-statistic: -14.497, p-value: 0.00000
Significant difference between suspicious and control groups.
----------------------------------------------------------------
comparing levenshtein_distance between df_smol_135m and df_smol_360m
mean (df_smol_135m) = 102.91240446796003, mean (df_smol_360m) = 97.20752498530277
t-statistic: 6.228, p-value: 0.00000
Significant difference between suspicious and control groups.
----------------------------------------------------------------
comparing rouge1_precision between df_smol_135m and df_smol_360m
mean (df_smol_135m) = 0.1045275028261191, mean (df_smol_360m) = 0.13916567407493322
t-statistic: -10.944, p-value: 0.00000
Significant difference between suspicious and control groups.
----------------------------------------------------------------
comparing rouge1_recall between df_smol_135m and df_smol_36

In [102]:
df_smol

,cos_sim_scores,levenshtein_distance,rouge1_precision,rouge1_recall,rouge1_f1,rouge2_precision,rouge2_recall,rouge2_f1,rougeL_precision,rougeL_recall,rougeL_f1,Min_10.0%_Prob,Min_20.0%_Prob,Min_30.0%_Prob,Min_40.0%_Prob,Min_50.0%_Prob
df_smol_135m vs df_smol_360m,"1, 0.2756, 0.3638","1, 102.9124, 97.2075","1, 0.1045, 0.1392","1, 0.1724, 0.2325","1, 0.1274, 0.1704","1, 0.0139, 0.024","1, 0.023, 0.0408","1, 0.0169, 0.0296","1, 0.082, 0.1104","1, 0.1355, 0.185","1, 0.1, 0.1353",0,0,0,"1, 7.8712, 7.0262","1, 7.0355, 6.2223"
df_smol_135m vs df_smol_1_7b,"1, 0.2756, 0.3941","1, 102.9124, 95.963","1, 0.1045, 0.1594","1, 0.1724, 0.2665","1, 0.1274, 0.1953","1, 0.0139, 0.037","1, 0.023, 0.0638","1, 0.0169, 0.0458","1, 0.082, 0.1288","1, 0.1355, 0.2161","1, 0.1, 0.158",0,0,"1, 8.8508, 7.0016","1, 7.8712, 6.1042","1, 7.0355, 5.3515"
df_smol_360m vs df_smol_1_7b,"1, 0.3638, 0.3941",0,"1, 0.1392, 0.1594","1, 0.2325, 0.2665","1, 0.1704, 0.1953","1, 0.024, 0.037","1, 0.0408, 0.0638","1, 0.0296, 0.0458","1, 0.1104, 0.1288","1, 0.185, 0.2161","1, 0.1353, 0.158",0,0,0,"1, 7.0262, 6.1042","1, 6.2223, 5.3515"


In [103]:
sus_con_pairs = [("df_gpt_neo_125m", "df_smol_135m"), ("df_opt_125m", "df_smol_135m")]

In [104]:
# sus vs. con (similar model sizes) comparison
def sus_con_wise_compare(pair_list):
    print(pair_list)
    df_model = pd.DataFrame()
    for pair in pair_list:
        score_sig = t_test(pair[0], pair[1], score_names)
        print(score_sig)
        df_model = pd.concat([df_model, pd.DataFrame(score_sig, index=[f"{pair[0]} vs {pair[1]}"])])
        print()
    return df_model


In [105]:
df_sus_con_compare = sus_con_wise_compare(sus_con_pairs)

[('df_gpt_neo_125m', 'df_smol_135m'), ('df_opt_125m', 'df_smol_135m')]
comparing cos_sim_scores between df_gpt_neo_125m and df_smol_135m
mean (df_gpt_neo_125m) = 0.2837392103285181, mean (df_smol_135m) = 0.275623557275188
t-statistic: 1.341, p-value: 0.18012
No significant difference detected.
----------------------------------------------------------------
comparing levenshtein_distance between df_gpt_neo_125m and df_smol_135m
mean (df_gpt_neo_125m) = 97.11934156378601, mean (df_smol_135m) = 102.91240446796003
t-statistic: -6.282, p-value: 0.00000
Significant difference between suspicious and control groups.
----------------------------------------------------------------
comparing rouge1_precision between df_gpt_neo_125m and df_smol_135m
mean (df_gpt_neo_125m) = 0.13017792478245246, mean (df_smol_135m) = 0.1045275028261191
t-statistic: 8.119, p-value: 0.00000
Significant difference between suspicious and control groups.
----------------------------------------------------------------

In [106]:
df_sus_con_compare.to_csv(f"t_test/{dataset_name}/sus_con_compare.csv")